In [ ]:
import requests as r
import os
from bs4 import BeautifulSoup as bs
import lxml
from urllib.parse import urlparse,parse_qs,urlencode,urlunparse

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    # 如果目标站点需要，可以加上 Cookie 或 Referer
    # "Referer": "https://example.com/",
    # "Cookie": "name=value; name2=value2"
}

def fetch_url(url,file):
    respond=r.get(url,headers=headers)
    if respond.status_code==200:
        with open(file, "w") as f:
            f.write(respond.text)
        print(f"File saved in {file}.")
    else:
        raise Exception(f"Failed to download file. Status code: {respond.status_code}")

url="https://wiki.biligame.com/blhx/首页"
file="index.html"

if not os.path.exists(file):
    fetch_url(url,file)

soup=bs(open("html/index.html"),"lxml")

a_list=soup.find_all("a",class_="menu-title-1")

bg=[_ for _ in a_list if "壁纸和插画" in _.text][0]

bg_url=url+bg.get("href")
bg_url

'https://wiki.biligame.com/blhx/blhx/%E5%BD%B1%E7%94%BB%E7%9B%B8%E5%85%B3'